In [1]:
import spacy
import feedparser
import textnets as tn
import pandas as pd
import string
from pyvis.network import Network
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import math
from bs4 import BeautifulSoup
import requests
import sys
from unicodedata import category
import progressbar
import csv

c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-pac

In [2]:
feed_urls = [
    "http://www.lemonde.fr/rss/une.xml",
    "https://www.bfmtv.com/rss/news-24-7/",
    "https://www.liberation.fr/rss/",
    "http://www.lefigaro.fr/rss/figaro_actualites.xml",
    "https://www.franceinter.fr/rss",
    "https://www.lexpress.fr/arc/outboundfeeds/rss/alaune.xml",
    "https://www.francetvinfo.fr/titres.rss",
    "https://www.la-croix.com/RSS",
    "http://tempsreel.nouvelobs.com/rss.xml",
    "http://www.lepoint.fr/rss.xml",
    "https://www.france24.com/fr/rss",
    "https://feeds.leparisien.fr/leparisien/rss",
    "https://www.ouest-france.fr/rss/une",
    "https://www.europe1.fr/rss.xml",
    "https://partner-feeds.20min.ch/rss/20minutes",
    "https://www.afp.com/fr/actus/afp_actualite/792,31,9,7,33/feed"
]

In [3]:
news_list = pd.DataFrame(columns=('title', 'summary'))

for feed_url in feed_urls:
    r = requests.get(feed_url)
    feed = BeautifulSoup(r.content, features='xml')
    #feed = feedparser.parse(feed_url)

    articles = feed.findAll('item')       
    for a in articles:
        title = BeautifulSoup(a.find('title').get_text(), "html").get_text()
        summary = ""
        if (a.find('description')):
            summary = BeautifulSoup(a.find('description').get_text(), "html").get_text()
        news_list.loc[len(news_list)] = [title, summary]

c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
c:\Users\anton\AppData\Local\Programs\Python\Python39\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this

In [4]:
news_list

,title,summary
0,Manifestation du 31 janvier contre la réforme ...,"Selon les autorités, 1,272 million de personne..."
1,Manifestations du 31 janvier : les images des ...,D’Arras à Marseille en passant par Toulouse et...
2,Manifestation du 31 janvier contre la réforme ...,Alors que les écarts entre les chiffres dénomb...
3,Calculez votre âge de départ à la retraite ava...,Le projet du gouvernement repousse l’âge légal...
4,Référendum sur les retraites : seule la motion...,Pour départager les deux motions référendaires...
...,...,...
531,"Le podcast ""Le Poison de Poutine"" récompensé p...","Le Poison de Poutine, premier podcast document..."
532,Franck Fife récompensé par le Prix Emilie-Pouc...,"Lors de la 45e édition du Dakar, le jury compo..."
533,Lutte contre la désinformation : l’AFP dévoile...,Forte du plus grand réseau de fact-checking au...
534,Eduardo Soteras récompensé par le prix UNICEF ...,Eduardo Soteras s’est vu remettre le prix UNIC...


In [53]:
def process_text(docs, lang='fr'):
    if (lang=='fr'):
        nlp = spacy.load('fr_core_news_sm')
    elif (lang=='en'):
        nlp = spacy.load('en_core_web_sm')

    widgets = [' [',
         progressbar.Timer(format= 'elapsed time: %(elapsed)s'),
         '] ',
           progressbar.Bar('*'),'[',
           progressbar.Percentage(), '] (',
           progressbar.ETA(), ') ',
          ]
 
    bar = progressbar.ProgressBar(max_value=len(docs),
                                widgets=widgets).start()
    
    
    # Utility functions
    punctuation_chars =  [
        chr(i) for i in range(sys.maxunicode)
        if category(chr(i)).startswith("P")
        ]
    stemmer = PorterStemmer()
    def tokenize(text):
        global count
        #parsed_text = [ch for ch in text]
        #for i, ch in enumerate(text):
        #    if (ch == "'" or ch == "’"):
        #        parsed_text[i - 1] = ""
        #    if (ch in [*string.punctuation, *punctuation_chars]):
        #        parsed_text[i] = ""
        #    else:
        #        parsed_text[i] = ch
        #text = "".join(parsed_text)
        count = count + 1
        bar.update(count)
        text = "".join(list(filter(lambda x: x not in [*string.punctuation, *punctuation_chars], text)))
        tokens = nltk.word_tokenize(text)
        words = list(filter(lambda x: x not in [stopwords.words('english') + stopwords.words('french')], tokens))
        return list(map(lambda x: x.lower(), words))

    def preprocess_text(documents):
        global count
        count = 0
        docs = list(map(lambda doc: tokenize(doc), documents))
        return docs
    
    # Clean and tokenize docs
    tokenized_docs = preprocess_text(docs)
    print('tk news')
    
    # Lemmanize docs
    def lemmanize(doc):
        doc = list(filter(lambda token: token.lemma_ not in nlp.Defaults.stop_words, doc))
        return list(map(lambda token: token.lemma_, doc))

    lemma_docs = list(map(lambda doc: lemmanize(nlp(" ".join(doc))), tokenized_docs))
    print('lemma news')
    def get_vocabulary_frequency(documents):
        vocabulary = dict()
        for doc in documents:
            for word in doc:
                print(word)
                if word in list(vocabulary.keys()):
                    vocabulary[word] += 1
                else:
                    vocabulary[word] = 1

        return vocabulary

    voc = get_vocabulary_frequency(lemma_docs)

    return lemma_docs, voc

In [35]:
def graphnet(docs, voc, min_freq=5, output_url='graph.html'):
    
    # Filter voc with min_freq
    filtered_voc = dict(filter(lambda elem: elem[1] > min_freq, voc.items()))
    print(sorted(filtered_voc.items(), key=lambda x: x[1], reverse=True)[:20])

    dict_voc_id = dict()
    for i, term in enumerate(filtered_voc):
        dict_voc_id[term] = i
    
    # List bigrams (edges)
    finder = nltk.BigramCollocationFinder.from_documents(docs)
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    bigrams = list(finder.score_ngrams(bigram_measures.raw_freq))
    bigrams = list(map(lambda x: x[0], bigrams))

    # Filter the bigrams with filtered_voc elements and replace by id
    bigrams = list(filter(lambda x: x[0] in filtered_voc.keys() and x[1] in filtered_voc.keys(), bigrams))
    bigrams = list(map(lambda x: (dict_voc_id[x[0]], dict_voc_id[x[1]]), bigrams))

    # Set sizes according to (log of doc freq)*freq
    sizes = list(filtered_voc.values())

    # Output network
    net = Network( bgcolor="#222222", font_color="white", height="800px")

    net.add_nodes(range(0, len(filtered_voc)), label=list(filtered_voc.keys()), size=sizes, title=list(filtered_voc.keys())) # node ids and labels = ["a", "b", "c", "d"]
    net.add_edges(bigrams)

    net.toggle_physics(True)
    net.save_graph(output_url)

    with open('nodes.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(['id', 'label', 'size'])

        # write multiple rows
        for i, term in enumerate(filtered_voc.keys()):
            writer.writerow([i, term, sizes[i]])
    
    with open('edges.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(['id', 'Source', 'Target'])

        # write multiple rows
        for i, edge in enumerate(bigrams):
            (s, t) = edge
            writer.writerow([i, s, t])

In [51]:
docs, voc = process_text(news_list['summary'], lang='fr')

 [elapsed time: 0:00:00] |                            |[  0%] (ETA:  --:--:--)
 [elapsed time: 0:00:00] |                            |[  0%] (ETA:   0:01:31)
 [elapsed time: 0:00:00] |                            |[  2%] (ETA:   0:00:08)
 [elapsed time: 0:00:00] |*                           |[  5%] (ETA:   0:00:05)
 [elapsed time: 0:00:00] |**                          |[  8%] (ETA:   0:00:03)
 [elapsed time: 0:00:00] |***                         |[ 11%] (ETA:   0:00:03)
 [elapsed time: 0:00:00] |***                         |[ 13%] (ETA:   0:00:03)
 [elapsed time: 0:00:00] |****                        |[ 16%] (ETA:   0:00:02)
 [elapsed time: 0:00:00] |*****                       |[ 20%] (ETA:   0:00:02)
 [elapsed time: 0:00:00] |******                      |[ 22%] (ETA:   0:00:02)
 [elapsed time: 0:00:00] |*******                     |[ 25%] (ETA:   0:00:02)
 [elapsed time: 0:00:00] |*******                     |[ 27%] (ETA:   0:00:02)
 [elapsed time: 0:00:00] |********                  

tk news
[['manifestation', 'du', '31', 'janvier', 'contre', 'la', 'réforme', 'des', 'retraites', 'en', 'direct', 'après', 'une', 'journée', 'de', 'mobilisation', 'record', 'les', 'syndicats', 'appellent', 'à', 'manifester', 'de', 'nouveau', 'les', '7', 'et', '11', 'février'], ['manifestations', 'du', '31', 'janvier', 'les', 'images', 'des', 'défilés', 'contre', 'la', 'réforme', 'des', 'retraites'], ['manifestation', 'du', '31', 'janvier', 'contre', 'la', 'réforme', 'des', 'retraites', 'comment', 'sont', 'comptabilisés', 'les', 'manifestants'], ['calculez', 'votre', 'âge', 'de', 'départ', 'à', 'la', 'retraite', 'avant', 'et', 'après', 'la', 'réforme'], ['référendum', 'sur', 'les', 'retraites', 'seule', 'la', 'motion', 'du', 'rn', 'sera', 'débattue', 'la', 'gauche', 'dénonce', 'une', 'décision', 'arbitraire'], ['réforme', 'des', 'retraites', 'les', 'gagnants', 'et', 'les', 'perdants', 'en', 'neuf', 'cas', 'pratiques'], ['guerre', 'en', 'ukraine', 'la', 'france', 'va', 'fournir', '12', 'n

In [52]:
graphnet(docs, voc, min_freq=5)

[('retraite', 57), ('réforme', 38), ('guerre', 30), ('ukraine', 26), ('contre', 25), ('france', 23), ('pari', 22), ('janvier', 21), ('février', 20), ('million', 18), ('dun', 17), ('faire', 17), ('31', 16), ('aller', 15), ('mobilisation', 13), ('direct', 12), ('journée', 12), ('kiev', 12), ('7', 11), ('manifestant', 11)]


In [12]:
data = pd.read_json('News_Category_Dataset_v3.json', lines=True)

In [13]:
data

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [81]:
docs_en, voc_en = process_text(data['headline'], lang='en')

 [elapsed time: 0:00:00] |                            |[  0%] (ETA:  --:--:--)


UnboundLocalError: local variable 'count' referenced before assignment

In [64]:
graphnet(docs_en, voc_en, min_freq=500)

 [elapsed time: 0:00:00] |                            |[  0%] (ETA:  --:--:--)


[('photo', 12364), ('trump', 11489), ('new', 9084), ('I', 7000), ('video', 6192), ('day', 5339), ('good', 5175), ('donald', 4842), ('woman', 4581), ('way', 4123), ('year', 3674), ('man', 3464), ('5', 3455), ('world', 3426), ('life', 3359), ('kid', 3217), ('10', 3128), ('find', 3091), ('people', 3055), ('love', 3014)]
697332
57621
[(216, 124), (400, 401), (25, 226), (216, 217), (133, 103), (135, 162), (137, 138), (244, 318), (206, 84), (43, 160), (25, 49), (208, 282), (425, 10), (235, 360), (145, 19), (389, 395), (391, 397), (91, 92), (261, 114), (157, 158)]
